In [64]:
import numpy as np
import seaborn as sns
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt


In [65]:
# get the data

concrete_dataframe = pd.read_csv("https://raw.githubusercontent.com/alexander-one/portfolio/main/Project%20-%20Concrete%20Strength%20ML/assets/Concrete_Data.csv")

In [66]:
# clean data
concrete_dataframe.dropna()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age (day),Concrete compressive strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


In [67]:
# prepare the data for linear regression
from sklearn import preprocessing

In [68]:
# standardize
scaled_concrete = preprocessing.StandardScaler().fit(concrete_dataframe)
transformed_concrete = scaled_concrete.transform(concrete_dataframe)

From the ReadMe with the data: 

Cement (component 1) -- quantitative -- kg in a m3 mixture -- Input Variable
Blast Furnace Slag (component 2) -- quantitative -- kg in a m3 mixture -- Input Variable
Fly Ash (component 3) -- quantitative -- kg in a m3 mixture -- Input Variable
Water (component 4) -- quantitative -- kg in a m3 mixture -- Input Variable
Superplasticizer (component 5) -- quantitative -- kg in a m3 mixture -- Input Variable
Coarse Aggregate (component 6) -- quantitative -- kg in a m3 mixture -- Input Variable
Fine Aggregate (component 7) -- quantitative -- kg in a m3 mixture -- Input Variable
Age -- quantitative -- Day (1~365) -- Input Variable
Concrete compressive strength -- quantitative -- MPa -- Output Variable 

In [69]:
# make sure it loaded
concrete_dataframe.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age (day),Concrete compressive strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [70]:
#looking to make sure the datatypes are all non-categorical
concrete_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Cement                          1030 non-null   float64
 1   Blast Furnace Slag              1030 non-null   float64
 2   Fly Ash                         1030 non-null   float64
 3   Water                           1030 non-null   float64
 4   Superplasticizer                1030 non-null   float64
 5   Coarse Aggregate                1030 non-null   float64
 6   Fine Aggregate                  1030 non-null   float64
 7   Age (day)                       1030 non-null   int64  
 8   Concrete compressive strength   1030 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 72.5 KB


In [71]:
concrete_dataframe = transformed_concrete

In [72]:
from sklearn.model_selection import train_test_split

In [73]:
# split the data into train and test
train_set, test_set = train_test_split(concrete_dataframe, test_size = 0.2, random_state = 42)

In [74]:
# data exploration without messing with the original train_set
concrete = train_set.copy()

In [75]:
#Pearson's r
corr_matrix = concrete.corr()

# remove redundancy in the heatmap about to come up
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

AttributeError: 'numpy.ndarray' object has no attribute 'corr'

In [ ]:
#produce a visual of the correlation between features
f, ax = plt.subplots(figsize=(11, 9))

cmap = sns.diverging_palette(230, 20, as_cmap=True)

sns.heatmap(corr_matrix, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
corr_matrix["Concrete compressive strength "].sort_values(ascending = False)

Let's take superplasticizer, age, water, fine aggregate, and compressive strength and see what the trends are. 

In [ ]:
from pandas.plotting import scatter_matrix

In [ ]:
attributes = ["Superplasticizer", "Water", "Age (day)", "Fine Aggregate", "Concrete compressive strength "]
scatter_matrix(concrete[attributes], figsize = (12,8))